In [1]:
#%load_ext autoreload
#%autoreload 2

from language import Transformer
from sae import Tracer, Visualizer
from sae.functions import compute_truncated_eigenvalues

import plotly.express as px
from tqdm import tqdm

import torch

device = "cpu"

In [2]:
torch.set_grad_enabled(False)
model = Transformer.from_pretrained("tdooms/ts-medium").to(device)

In [3]:
model.generate("Hello world!", max_length=30)

'hello world! said sam. samantha. she loved animals and exploring. mom and soothy said? she wanted sally to prepare and purple rain'

In [4]:
from datasets import load_dataset, load_from_disk

dataset = load_from_disk("ts-tokenized-final").select_columns(["input_ids"]).with_format("torch")

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [ ]:
#Visualizer.compute_max_activations(model, dataset=dataset['train'], in_batch=16)

In [14]:
# We set up a Tracer object, which is a utility class to find interesting interactions between two SAEs around an MLP.
# Let's inspect a middle layer.
tracer = Tracer(model, layer=5, inp=dict(expansion=4), out=dict(expansion=4))

# We then create a visualizer for both SAEs.
# Implementation-wise, this queries some pre-computed max-activations and shows them in a nice format.
inp_vis = Visualizer(model, tracer.inp, dataset=dataset['train'])
out_vis = Visualizer(model, tracer.out, dataset=dataset['train'])

In [15]:
# Compute output features whose top eigenvalues are high, likely indicating some interesting structure.
eigenvals = tracer.compute(compute_truncated_eigenvalues, project=False, k=64)
vals, idxs = eigenvals.topk(15)

# Plot the cosine similarity between these features to see if any are related.
dirs = tracer.out.w_dec.weight[:, idxs]
sims = torch.cosine_similarity(dirs[..., None], dirs[:, None], dim=0)

# Visualize them nicely.
labels = [f"{i}" for i in idxs.cpu()]
px.imshow(sims.cpu(), color_continuous_scale="RdBu", color_continuous_midpoint=0, x=labels, y=labels)

100%|██████████| 64/64 [00:28<00:00,  2.23it/s]


In [16]:
out_vis(1260, 100, k=10)

feature 1260
10.8:  hermommytookis##a##be##llaoutsideandthereinthebrightsunshinewasabeautifulparkwithabran##dnewplayground!is##a##be##llawas
9.2 :  hermommy.itwasagloomyevening,andtheskywasdark.lilylovedtoplayinthepark,buttodayshefeltsad."
7.7 :  hermommyanddaddywereworriedbecausetheydidn'thaveenoughmoneytobuyeverythingshewanted.butthen,adeliver##ymancametotheir
8.9 :  shouldhavecheckedthatitworkedbeforewechoseit.jackrealisedthatitwasagoodideatocheckbeforeyouchoosesomething.hesaidtojill
10.3:  frustratedbecausehereallywantedtoseewhatwasinthenewpartoftheocean.heaskedhisfriendsiftheycouldcomewithhim,butthey
9.7 :  [EOS][EOS][EOS][EOS][EOS][EOS][EOS][EOS][EOS][EOS][EOS][EOS][EOS][EOS][EOS][EOS][EOS][EOS]
8.3 :  fish."iliketoswimfastandjumphighintheopenwater.itisfun!"theboatandthebigfishbecamefriends.
11.3:  toshareit.shehopedherplantlikedshow-and-telltoo.
10.8:  littleboywasscaredandra##isedhishandstothesky.hemadeawish.hewishedhehadaguntoshoot.thebirdwas
10.5:  haveputitinthepooltomakeite